In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Number of GPUs available: {len(gpus)}")
    for i, gpu in enumerate(gpus):
        print(f"GPU {i}: {gpu}")
else:
    print("No GPU available")

No GPU available


In [ ]:
# Load dataset with proper split
train_dataset, val_dataset, test_dataset = tfds.load(
    'patch_camelyon',
    split=['train[:30%]', 'test[:50%]', 'test[50%:]'],
    as_supervised=True
)

# Check the number of examples in each dataset
print(f"Number of training examples: {len(train_dataset)}")
print(f"Number of validation examples: {len(val_dataset)}")
print(f"Number of test examples: {len(test_dataset)}")

Number of training examples: 78643
Number of validation examples: 16384
Number of test examples: 16384


In [ ]:
def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)  # Scale pixel values between 0 and 1
    return image, label

train_dataset = (train_dataset.map(preprocess)
                 .shuffle(1000)
                 .batch(32)
                 .cache()
                 .prefetch(tf.data.experimental.AUTOTUNE))

val_dataset = (val_dataset.map(preprocess)
               .batch(32)
               .cache()
               .prefetch(tf.data.experimental.AUTOTUNE))

test_dataset = (test_dataset.map(preprocess)
                .batch(32)
                .cache()
                .prefetch(tf.data.experimental.AUTOTUNE))


In [ ]:
# Define the custom CNN model using KerasTuner HyperModel
def build_model(hp):
    model = models.Sequential()

    model.add(layers.Conv2D(hp.Choice('conv1_filters', [32, 64]), (3, 3), activation='relu', input_shape=(96, 96, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(hp.Float('dropout1', 0.2, 0.4, step=0.1)))

    model.add(layers.Conv2D(hp.Choice('conv2_filters', [64, 128]), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(hp.Float('dropout2', 0.2, 0.4, step=0.1)))

    model.add(layers.Conv2D(hp.Choice('conv3_filters', [128, 256]), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(hp.Float('dropout3', 0.3, 0.5, step=0.1)))

    model.add(layers.Conv2D(hp.Choice('conv3_filters', [256, 512]), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(hp.Float('dropout3', 0.3, 0.5, step=0.1)))

    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', 256, 512, step=128), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout_dense', 0.3, 0.5, step=0.1)))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='kt_search',
    project_name='pcam_cnn_tuning'
)

Reloading Tuner from kt_search/pcam_cnn_tuning/tuner0.json


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

tuner.search(train_dataset, epochs=4, validation_data=val_dataset, callbacks=[early_stopping])

Trial 10 Complete [00h 19m 26s]
val_accuracy: 0.818115234375

Best val_accuracy So Far: 0.8559548258781433
Total elapsed time: 02h 29m 13s


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
test_loss, test_accuracy = best_model.evaluate(test_dataset)
print(f"Best Model Test Accuracy: {test_accuracy:.2f}")

512/512 [==============================] - 10s 20ms/step - loss: 0.5132 - accuracy: 0.7654
Best Model Test Accuracy: 0.77


In [ ]:
best_model.summary()
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:", best_hyperparameters.values)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 94, 94, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 47, 47, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 47, 47, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 45, 45, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 22, 22, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 22, 22, 64)        0